In [2]:
import io, os, re, sys
import numpy as np
import pandas as pd
import pytesseract
import fitz
import PyPDF2

from dotenv import load_dotenv
from PIL import Image

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:
load_dotenv()

True

In [5]:
llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",
            google_api_key=os.getenv("GOOGLE_API_KEY"), 
            temperature=0.1
        )
embeddings = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001",
            google_api_key=os.getenv("GOOGLE_API_KEY")
        ) 

In [6]:
print(sys.version)
print(sys.executable)

3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:53:34) [Clang 16.0.6 ]
/Users/haneulkim/Desktop/Projects/RAG_experiments/venv_rag/bin/python


In [7]:
os.listdir('data/')

['LG_Energy_Soltuion_Valueup_Plan_ENG.pdf',
 '252Q_LGES_Audit_Report_CONFS_en.pdf',
 '.ipynb_checkpoints',
 '2025_1Q_LGES_Audit_Report_CONFS_en.pdf',
 'processed']

# Extract data from PDF

In [8]:
q1_report_path = "data/2025_1Q_LGES_Audit_Report_CONFS_en.pdf"
docs = fitz.open(q1_report_path)
print(f"total pages: {len(docs)}")

total pages: 65


In [9]:
print(docs, type(docs))
print(docs[0], type(docs[0]))

Document('data/2025_1Q_LGES_Audit_Report_CONFS_en.pdf') <class 'pymupdf.Document'>
page 0 of data/2025_1Q_LGES_Audit_Report_CONFS_en.pdf <class 'pymupdf.Page'>


In [10]:
def is_scanned_page(page):
    """
    If pages are covered with only images, run OCR.

    ??? What if half is filled with images and half with text?
    """
    text = page.get_text().strip()
    images = page.get_images()
    return len(images) > 0 and len(text) < 25 

def extract_text_ocr(page):
    pix = page.get_pixmap()
    img_data = pix.tobytes("png")
    img = Image.open(io.BytesIO(img_data))

    # text extraction using OCR
    text = pytesseract.image_to_string(img)
    return text

def clean_text(text):
    """Clean extracted text"""
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters but keep punctuation
    text = re.sub(r'[^\w\s\-.,;:!?()]', '', text)
    return text.strip()

In [13]:
full_text = ""
for pg_i, page in enumerate(docs):
    if is_scanned_page(page):
        ocr_text = extract_text_ocr(page)
        if ocr_text.strip():
            page_text = ocr_text
        else:
            print(f"--------OCR failed------- at page:{pg_i}")
    else:
        page_text = page.get_text()
        
    page_text = clean_text(page_text)
    full_text += f"\n--- Page {pg_i + 1} ---\n{page_text}\n"
    
with open('data/processed/2025_1Q_LGES_Audit_Report_CONFS_en_fulltext.txt', 'w') as f:
   f.write(full_text) 

In [ ]:
# ??? Extract Table of content: So it's information can be leveraged somewher(ex: chunk by chapter, etc..)
# table_of_contents = doc[1].get_text()

# ??? Better way to extract table? and standardize schema across various reports from diff countries.
# table = doc[5].get_text() + "\n" + doc[6].get_text()